In [2]:
import cv2
import torch
import numpy as np
import os
from torchvision import transforms, models
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
from scipy.spatial.distance import cosine

# --------------------------------
# CONFIGURATIONS & LOADING MODELS
# --------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MTCNN for Face Detection (WITH PADDING)
mtcnn = MTCNN(keep_all=True, device=device)

# Load Face Recognition Model (FaceNet)
facenet = InceptionResnetV1(pretrained="casia-webface").eval().to(device)

# Load Fine-tuned Emotion Detection Model (EfficientNet-B2)
emotion_classes = ["Anger", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]
model_save_path = "efficientnet_b2_emotion_model.pth"

def load_emotion_model(model_path: str) -> torch.nn.Module:
    print("Loading emotion detection model...")
    model = models.efficientnet_b2(pretrained=False)
    # Adjust first layer to match standard EfficientNet-B2 shape
    model.features[0][0] = torch.nn.Conv2d(
        3, 32, kernel_size=3, stride=2, padding=1, bias=False
    )
    model.classifier = torch.nn.Sequential(
        torch.nn.Dropout(p=0.4),
        torch.nn.Linear(model.classifier[1].in_features, len(emotion_classes)),
    )
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    print("Emotion model loaded!")
    return model

emotion_model = load_emotion_model(model_save_path)

# Reference Face Directory
reference_folder = "./reference-face-frames-collect"

# File where we store/load the average embedding
embedding_file = "average_embedding.npy"

# Padding for face recognition (NOT for emotion detection)
padding_ratio = 0.3  

# Higher similarity threshold for better recognition
similarity_threshold = 0.6  

# Quit Button Configuration
quit_program = False
quit_button_coords = (10, 10, 100, 50)  # (x1, y1, x2, y2)

# -------------------------
# FUNCTIONS & LOGIC
# -------------------------
def get_face_embedding(image: Image.Image) -> np.ndarray or None:
    """
    Runs MTCNN on the given PIL image, crops the face, and obtains a 512-dim
    embedding from FaceNet. Returns None if no face is detected or invalid shape.
    """
    img_cropped = mtcnn(image)
    if img_cropped is None:
        return None
    if img_cropped.ndim == 3:
        img_cropped = img_cropped.unsqueeze(0)
    img_cropped = img_cropped.to(device)
    embedding = facenet(img_cropped).detach().cpu().numpy().flatten()
    return embedding if embedding.shape[0] == 512 else None

def load_reference_embeddings(reference_dir: str) -> np.ndarray or None:
    """
    Reads all images from the specified directory, computes their FaceNet embeddings,
    and returns the average embedding as a NumPy array. Returns None if no valid
    embeddings are found.
    """
    embeddings = []
    for file in os.listdir(reference_dir):
        if file.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(reference_dir, file)
            image = Image.open(image_path)
            embedding = get_face_embedding(image)
            if embedding is not None:
                embeddings.append(embedding)

    if len(embeddings) == 0:
        return None
    
    # Compute the average embedding
    avg_embedding = np.mean(embeddings, axis=0)
    return avg_embedding

# -------------------------
# LOAD OR COMPUTE EMBEDDING
# -------------------------
if os.path.exists(embedding_file):
    # If we already have a saved average embedding, load it
    reference_embedding = np.load(embedding_file)
    print(f"[INFO] Loaded existing average embedding from '{embedding_file}'.")
else:
    # Otherwise, compute from reference images and save
    reference_embedding = load_reference_embeddings(reference_folder)
    if reference_embedding is None:
        print("No valid reference faces found! Exiting...")
        exit()
    else:
        np.save(embedding_file, reference_embedding)
        print(f"[INFO] Computed and saved average embedding to '{embedding_file}'.")

# -------------
# EMOTION DETECTION
# -------------
def preprocess_face_for_emotion(face_image: np.ndarray) -> torch.Tensor:
    """
    Convert the face image (BGR NumPy array) to a PyTorch tensor with the
    transformations required for the emotion model.
    """
    transform = transforms.Compose([
        transforms.Resize((260, 260)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])
    pil_image = Image.fromarray(cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB))
    return transform(pil_image).unsqueeze(0).to(device)

def predict_emotion(face_image: np.ndarray) -> (str, float):
    """
    Given a cropped face image (BGR), returns the predicted emotion class
    and its confidence score.
    """
    processed_image = preprocess_face_for_emotion(face_image)
    with torch.no_grad():
        outputs = emotion_model(processed_image)
        probabilities = torch.softmax(outputs, dim=1)
        predicted_class_idx = torch.argmax(probabilities, dim=1).item()
        return emotion_classes[predicted_class_idx], probabilities[0, predicted_class_idx].item()

# -------------
# FACE RECOGNITION
# -------------
def recognize_face(face_image: np.ndarray) -> (bool, float):
    """
    Given a face image (BGR NumPy array), compute the embedding and compare it
    against the loaded average embedding using cosine similarity.
    Returns (is_match, similarity).
    """
    # Convert BGR NumPy array to PIL for embedding
    face_embedding = get_face_embedding(Image.fromarray(face_image))
    if face_embedding is None:
        return False, -1

    similarity = 1 - cosine(face_embedding, reference_embedding)
    return (similarity > similarity_threshold), similarity

# -------------
# QUIT BUTTON
# -------------
def check_quit_click(event, x, y, flags, param):
    """
    Mouse callback to detect if the user clicks the 'Quit' button area.
    """
    global quit_program
    if event == cv2.EVENT_LBUTTONDOWN:
        if (quit_button_coords[0] <= x <= quit_button_coords[2] and
            quit_button_coords[1] <= y <= quit_button_coords[3]):
            quit_program = True

# -------------
# LIVE RECOGNITION
# -------------
def run_live_recognition():
    """
    Opens the default webcam, runs real-time face + emotion detection.
    Draws bounding boxes and displays the recognized user's emotion if matched.
    """
    global quit_program
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Cannot access webcam.")
        return

    cv2.namedWindow("Live Face Recognition & Emotion Detection")
    cv2.setMouseCallback("Live Face Recognition & Emotion Detection", check_quit_click)

    print("Starting real-time face recognition & emotion detection...")

    while not quit_program:
        ret, frame = cap.read()
        if not ret:
            print("Error: Cannot read frame from webcam.")
            break

        frame_height, frame_width, _ = frame.shape
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        boxes, _ = mtcnn.detect(rgb_frame)

        if boxes is not None:
            for box in boxes:
                x1, y1, x2, y2 = [int(coord) for coord in box]

                # Add padding for better recognition
                box_width = x2 - x1
                box_height = y2 - y1
                pad_w = int(box_width * padding_ratio)
                pad_h = int(box_height * padding_ratio)

                x1_pad = max(0, x1 - pad_w)
                y1_pad = max(0, y1 - pad_h)
                x2_pad = min(frame_width, x2 + pad_w)
                y2_pad = min(frame_height, y2 + pad_h)

                face_image = frame[y1_pad:y2_pad, x1_pad:x2_pad]  # For recognition
                cropped_face = frame[y1:y2, x1:x2]                 # For emotion

                if cropped_face.shape[0] > 0 and cropped_face.shape[1] > 0:
                    is_match, similarity = recognize_face(face_image)
                    if is_match:
                        emotion, confidence = predict_emotion(cropped_face)
                        label = f"YOU - {emotion} ({confidence:.2f})"
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(frame, label, (x1, y1 - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                                    (255, 255, 255), 2)

        # Draw Quit Button
        cv2.rectangle(frame,
                      (quit_button_coords[0], quit_button_coords[1]),
                      (quit_button_coords[2], quit_button_coords[3]),
                      (0, 0, 255), -1)
        cv2.putText(frame, "Quit",
                    (quit_button_coords[0] + 10, quit_button_coords[1] + 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        cv2.imshow("Live Face Recognition & Emotion Detection", frame)

        # Press 'q' to quit (in addition to the on-screen button)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Quitting...")
            break

    cap.release()
    cv2.destroyAllWindows()

# -------------
# MAIN
# -------------
if __name__ == "__main__":
    run_live_recognition()


100%|██████████| 111M/111M [00:22<00:00, 5.23MB/s] 


Loading emotion detection model...


c:\Users\Tuf\anaconda3\envs\dpsnBKP\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Tuf\anaconda3\envs\dpsnBKP\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Emotion model loaded!
[INFO] Computed and saved average embedding to 'average_embedding.npy'.
Starting real-time face recognition & emotion detection...
